In [4]:
# RAG HW
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS 
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
  temperature = 0.1
)

memory = ConversationBufferMemory(return_messages=True)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
  separator="\n",
  chunk_size=600,
  chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
  ("system", "You are helpful assistant. Answer questions using onlythe following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
  ("human", "{question}")
])

chain = {"context":retriever,"question":RunnablePassthrough()} | prompt | llm
chaintomemeory = chain | memory.save_context({"input":"{question}"},{"output":RunnablePassthrough()})

chaintomemeory.invoke("Is Aaronson guilty?")


AIMessage(content='Victory Mansion is a building where Winston Smith resides. It is described as having glass doors that let in gritty dust, a hallway that smells of boiled cabbage and old rag mats, and a poster of a large face with the caption "BIG BROTHER IS WATCHING YOU." The flat where Winston lives is on the seventh floor, and the building has a faulty lift due to the electricity being cut off during daylight hours as part of an economy drive. The building is located in London, the chief city of Airstrip One, which is one of the provinces of Oceania.')

In [ ]:
chain.invoke("What message did he write on the table?")

In [ ]:
chain.invoke("Who is Julia?")